# **1. Install**

## 1.1. Miniconda

In [1]:
# 1 Miniconda
!wget -c https://repo.anaconda.com/miniconda/Miniconda3-py37_4.12.0-Linux-x86_64.sh
!chmod +x Miniconda3-py37_4.12.0-Linux-x86_64.sh
!bash ./Miniconda3-py37_4.12.0-Linux-x86_64.sh -b -f -p /usr/local


import sys
sys.path.append('/usr/local/lib/python3.7/site-packages/')

# 2. Obabel
!conda install -c conda-forge openbabel
import openbabel
from openbabel import openbabel as ob
from openbabel import pybel

# 3. py3dmol

!conda install -c conda-forge py3dmol
import py3Dmol


# 4. RDKit
!pip install rdkit-pypi
from rdkit import*
# 5. Smina
!wget https://downloads.sourceforge.net/project/smina/smina.static
!chmod u+x smina.static
!./smina.static --help


# 6. Mgltools
!conda install -c bioconda mgltools -y

--2023-05-13 08:27:21--  https://repo.anaconda.com/miniconda/Miniconda3-py37_4.12.0-Linux-x86_64.sh
Resolving repo.anaconda.com (repo.anaconda.com)... 104.16.130.3, 104.16.131.3, 2606:4700::6810:8303, ...
Connecting to repo.anaconda.com (repo.anaconda.com)|104.16.130.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 104996770 (100M) [application/x-sh]
Saving to: ‘Miniconda3-py37_4.12.0-Linux-x86_64.sh’

Miniconda3-py37_4.1 100%[===================>] 100.13M   215MB/s    in 0.5s    

2023-05-13 08:27:22 (215 MB/s) - ‘Miniconda3-py37_4.12.0-Linux-x86_64.sh’ saved [104996770/104996770]

PREFIX=/usr/local
Unpacking payload ...
Solving environment: / - \ | / - \ | / done

## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - _libgcc_mutex==0.1=main
    - _openmp_mutex==4.5=1_gnu
    - brotlipy==0.7.0=py37h27cfd23_1003
    - ca-certificates==2022.3.29=h06a4308_1
    - certifi==2021.10.8=py37h06a4308_2
    - cffi==1.

# **2. Prepare receptor and ligand**

In [11]:
!wget http://files.rcsb.org/download/3ERK.pdb
!grep ATOM 3ERK.pdb > rec.pdb
!grep SB4 3ERK.pdb > lig.pdb

mol= [m for m in pybel.readfile(filename='lig.pdb',format='pdb')][0]
mol.addh()
out=pybel.Outputfile(filename='lig_H.mol2',format='mol2',overwrite=True)
out.write(mol)
out.close()


!python2 "/usr/local/MGLToolsPckgs/AutoDockTools/Utilities24/prepare_receptor4.py" -r rec.pdb
!python2 "/usr/local/MGLToolsPckgs/AutoDockTools/Utilities24/prepare_ligand4.py" -l lig_H.mol2 -o lig_H.pdbqt

--2023-05-13 08:35:58--  http://files.rcsb.org/download/3ERK.pdb
Resolving files.rcsb.org (files.rcsb.org)... 132.249.210.222
Connecting to files.rcsb.org (files.rcsb.org)|132.249.210.222|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/octet-stream]
Saving to: ‘3ERK.pdb.1’

3ERK.pdb.1              [ <=>                ] 270.37K  --.-KB/s    in 0.1s    

2023-05-13 08:35:58 (2.26 MB/s) - ‘3ERK.pdb.1’ saved [276858]

adding gasteiger charges to peptide


# **3. Simple Docking**

In [22]:
!./smina.static -r rec.pdbqt -l lig_H.pdbqt --autobox_ligand lig.pdb -o smina.sdf --seed 42 --cpu -1 --exhaustiveness 32 --num_modes 500 --log smina.txt

   _______  _______ _________ _        _______ 
  (  ____ \(       )\__   __/( (    /|(  ___  )
  | (    \/| () () |   ) (   |  \  ( || (   ) |
  | (_____ | || || |   | |   |   \ | || (___) |
  (_____  )| |(_)| |   | |   | (\ \) ||  ___  |
        ) || |   | |   | |   | | \   || (   ) |
  /\____) || )   ( |___) (___| )  \  || )   ( |
  \_______)|/     \|\_______/|/    )_)|/     \|


smina is based off AutoDock Vina. Please cite appropriately.

Weights      Terms
-0.035579    gauss(o=0,_w=0.5,_c=8)
-0.005156    gauss(o=3,_w=2,_c=8)
0.840245     repulsion(o=0,_c=8)
-0.035069    hydrophobic(g=0.5,_b=1.5,_c=8)
-0.587439    non_dir_h_bond(g=-0.7,_b=0,_c=8)
1.923        num_tors_div

Using random seed: 42

0%   10   20   30   40   50   60   70   80   90   100%
|----|----|----|----|----|----|----|----|----|----|
***************************************************

mode |   affinity | dist from best mode
     | (kcal/mol) | rmsd l.b.| rmsd u.b.
-----+------------+----------+----------
1       

# **4. Analysis**

## 4.1. Visualization

In [23]:
import py3Dmol
v = py3Dmol.view()
v.addModel(open('rec.pdb').read())
v.setStyle({'cartoon':{},'stick':{'radius':.1}})
v.addModel(open('lig.pdb').read())
v.setStyle({'model':1},{'stick':{'colorscheme':'redCarbon','radius':.125}})
v.addModelsAsFrames(open('smina.sdf').read())
v.setStyle({'model':2},{'stick':{'colorscheme':'greenCarbon'}})
v.animate({'interval':1000})
v.zoomTo({'model':1})
v.rotate(90)

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [24]:
from rdkit import Chem
view = py3Dmol.view()
view.removeAllModels()
view.setViewStyle({'style':'outline','color':'black','width':0.1})

view.addModel(open('rec.pdb','r').read(),format='pdb')
Prot=view.getModel()
Prot.setStyle({'cartoon':{'arrows':True, 'tubes':True, 'style':'oval', 'color':'white'}})
view.addSurface(py3Dmol.VDW,{'opacity':0.6,'color':'white'})


view.addModel(open('lig.pdb','r').read(),format='pdb')
ref_m = view.getModel()
ref_m.setStyle({},{'stick':{'colorscheme':'magentaCarbon','radius':0.2}})


results=Chem.SDMolSupplier('smina.sdf')

p=Chem.MolToMolBlock(results[0],False)

print('Reference: Magenta | Smina Pose: Cyan')
print ('Score: {}'.format(results[0].GetProp('minimizedAffinity')))

view.addModel(p,'mol')
x = view.getModel()
x.setStyle({},{'stick':{'colorscheme':'cyanCarbon','radius':0.2}})

view.zoomTo()
view.show()

[08:49:40] Explicit valence for atom # 6 N, 4, is greater than permitted
[08:49:40] ERROR: Could not sanitize molecule ending on line 64
[08:49:40] ERROR: Explicit valence for atom # 6 N, 4, is greater than permitted


ArgumentError: ignored

In [ ]:
results[0]

## 4.2. Useful functions

In [ ]:
import py3Dmol
from openbabel import pybel

from rdkit import Chem
from rdkit.Chem import AllChem,rdFMCS, rdMolAlign

def dok_to_sdf (dok_file=None,output=None):

    """
    dok_to_sdf ( dok_file=None, output=None )
    params:
    dok_file: str or path-like ; dok file from ledock docking
    output: str or path-like ; outfile from ledock docking, extension must be sdf
   """
    out=pybel.Outputfile(filename=output,format='sdf',overwrite=True)

    with open(dok_file, 'r') as f:
        doc=[line for line in f.readlines()]
    
    doc=[line.replace(line.split()[2],line.split()[2].upper()) if 'ATOM' in line else line for line in doc]
    
    start=[index for (index,p) in enumerate(doc) if 'REMARK Cluster' in p]
    finish=[index-1 for (index,p) in enumerate(doc) if 'REMARK Cluster' in p]
    finish.append(len(doc))

    interval=list(zip(start,finish[1:]))
    for num,i in enumerate(interval):
        block = ",".join(doc[i[0]:i[1]]).replace(',','')

        m=pybel.readstring(format='pdb',string=block)
        
        m.data.update({'Pose':m.data['REMARK'].split()[4]})
        m.data.update({'Score':m.data['REMARK'].split()[6]})
        del m.data['REMARK']

        out.write(m)

    out.close()
  

def pdbqt_to_sdf(pdbqt_file=None,output=None):

    results = [m for m in pybel.readfile(filename=pdbqt_file,format='pdbqt')]
    out=pybel.Outputfile(filename=output,format='sdf',overwrite=True)
    for pose in results:

        pose.data.update({'Pose':pose.data['MODEL']})
        pose.data.update({'Score':pose.data['REMARK'].split()[2]})
        del pose.data['MODEL'], pose.data['REMARK'], pose.data['TORSDO']

        out.write(pose)
    out.close()

def get_inplace_rmsd (ref,target):
    
    r=rdFMCS.FindMCS([ref,target])
    
    a=ref.GetSubstructMatch(Chem.MolFromSmarts(r.smartsString))
    b=target.GetSubstructMatch(Chem.MolFromSmarts(r.smartsString))   
    amap=list(zip(a,b))
    
    distances=[]
    for atomA, atomB in amap:
        pos_A=ref.GetConformer().GetAtomPosition (atomA)
        pos_B=target.GetConformer().GetAtomPosition (atomB)
        coord_A=np.array((pos_A.x,pos_A.y,pos_A.z))
        coord_B=np.array ((pos_B.x,pos_B.y,pos_B.z))
        dist_numpy = np.linalg.norm(coord_A-coord_B)        
        distances.append(dist_numpy)
        
    rmsd=math.sqrt(1/len(distances)*sum([i*i for i in distances]))
    
    return rmsd

## 4.2. RMSD calculation

In [ ]:
import os

# The powerful libraries from rdkit I will use:
from rdkit import Chem
from rdkit.Chem import rdFMCS,AllChem, Draw, PandasTools
from rdkit.Chem.Draw import DrawingOptions

import py3Dmol

import numpy as np
import math
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns

In [ ]:
results = [m for m in pybel.readfile(filename='lig.pdbqt',format='pdbqt')]
out=pybel.Outputfile(filename='lig.sdf',format='sdf',overwrite=True)

for pose in results:

    out.write(pose)
    out.close()

In [ ]:
lig = PandasTools.LoadSDF('lig.sdf')
Reference = lig['ROMol'][0]
Reference.SetProp('_Name','Ref')
Reference

In [ ]:
pdbqt_to_sdf(pdbqt_file='smina.pdbqt',output='smina.sdf')

In [ ]:
SminaPoses=PandasTools.LoadSDF('smina.sdf')
SminaPoses.head()

In [ ]:
SminaPoses.head()

In [ ]:
SminaPoses['ID']=["A_"+SminaPoses.loc[i,'Pose'] for i in SminaPoses.index]
SminaPoses['RMSD']=[get_inplace_rmsd(ref=Reference,target=SminaPoses.loc[i,"ROMol"]) for i in SminaPoses.index]
SminaPoses['Software']=["Smina" for i in SminaPoses.index]

SminaPoses.head()

# **5. Advanced docking**

## 5.1. Whole protein docking (Blind docking)

In [16]:
!./smina.static -r rec.pdbqt -l lig_H.pdbqt --autobox_ligand rec.pdb -o wholedocked.sdf --seed 42 --exhaustiveness 32 --num_modes 200 --log wsb4.txt

   _______  _______ _________ _        _______ 
  (  ____ \(       )\__   __/( (    /|(  ___  )
  | (    \/| () () |   ) (   |  \  ( || (   ) |
  | (_____ | || || |   | |   |   \ | || (___) |
  (_____  )| |(_)| |   | |   | (\ \) ||  ___  |
        ) || |   | |   | |   | | \   || (   ) |
  /\____) || )   ( |___) (___| )  \  || )   ( |
  \_______)|/     \|\_______/|/    )_)|/     \|


smina is based off AutoDock Vina. Please cite appropriately.

Weights      Terms
-0.035579    gauss(o=0,_w=0.5,_c=8)
-0.005156    gauss(o=3,_w=2,_c=8)
0.840245     repulsion(o=0,_c=8)
-0.035069    hydrophobic(g=0.5,_b=1.5,_c=8)
-0.587439    non_dir_h_bond(g=-0.7,_b=0,_c=8)
1.923        num_tors_div

Using random seed: 42

0%   10   20   30   40   50   60   70   80   90   100%
|----|----|----|----|----|----|----|----|----|----|
***************************************************

mode |   affinity | dist from best mode
     | (kcal/mol) | rmsd l.b.| rmsd u.b.
-----+------------+----------+----------
1       

In [17]:
import py3Dmol
v = py3Dmol.view()
v.addModel(open('rec.pdb').read())
v.setStyle({'cartoon':{},'stick':{'radius':.1}})
v.addModel(open('lig.pdb').read())
v.setStyle({'model':1},{'stick':{'colorscheme':'redCarbon','radius':.125}})
v.addModelsAsFrames(open('wholedocked.sdf').read())
v.setStyle({'model':2},{'stick':{'colorscheme':'greenCarbon'}})
v.animate({'interval':1000})
v.zoomTo({'model':1})
v.rotate(90)

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

## 5.2. Flexible docking

 Use **flexdist** arg to set all side chains within specified distance to **flexdist_ligand** to flexible

In [27]:
!./smina.static -r rec.pdbqt -l lig_H.pdbqt --autobox_ligand lig_H.pdbqt --seed 42 -o flexdocked.sdf --flexdist 4 --flexdist_ligand lig_H.pdbqt --out_flex flexout.pdb

   _______  _______ _________ _        _______ 
  (  ____ \(       )\__   __/( (    /|(  ___  )
  | (    \/| () () |   ) (   |  \  ( || (   ) |
  | (_____ | || || |   | |   |   \ | || (___) |
  (_____  )| |(_)| |   | |   | (\ \) ||  ___  |
        ) || |   | |   | |   | | \   || (   ) |
  /\____) || )   ( |___) (___| )  \  || )   ( |
  \_______)|/     \|\_______/|/    )_)|/     \|


smina is based off AutoDock Vina. Please cite appropriately.

Weights      Terms
-0.035579    gauss(o=0,_w=0.5,_c=8)
-0.005156    gauss(o=3,_w=2,_c=8)
0.840245     repulsion(o=0,_c=8)
-0.035069    hydrophobic(g=0.5,_b=1.5,_c=8)
-0.587439    non_dir_h_bond(g=-0.7,_b=0,_c=8)
1.923        num_tors_div

Flexible residues: A:37 A:51 A:52 A:82 A:101 A:102 A:103 A:104 A:105 A:106 A:109 A:112 A:151 A:154
Using random seed: 42

0%   10   20   30   40   50   60   70   80   90   100%
|----|----|----|----|----|----|----|----|----|----|
***************************************************

mode |   affinity | dist from b

Use **flexres** arg for flexible side chains specified by comma separated list of chain:resid

In [28]:
!./smina.static -r rec.pdbqt -l lig_H.pdbqt --autobox_ligand lig_H.pdbqt --seed 42 -o flexdocked2.sdf --exhaustiveness 16 --flexres A:52,A:103 --out_flex flexout2.sdf

   _______  _______ _________ _        _______ 
  (  ____ \(       )\__   __/( (    /|(  ___  )
  | (    \/| () () |   ) (   |  \  ( || (   ) |
  | (_____ | || || |   | |   |   \ | || (___) |
  (_____  )| |(_)| |   | |   | (\ \) ||  ___  |
        ) || |   | |   | |   | | \   || (   ) |
  /\____) || )   ( |___) (___| )  \  || )   ( |
  \_______)|/     \|\_______/|/    )_)|/     \|


smina is based off AutoDock Vina. Please cite appropriately.

Weights      Terms
-0.035579    gauss(o=0,_w=0.5,_c=8)
-0.005156    gauss(o=3,_w=2,_c=8)
0.840245     repulsion(o=0,_c=8)
-0.035069    hydrophobic(g=0.5,_b=1.5,_c=8)
-0.587439    non_dir_h_bond(g=-0.7,_b=0,_c=8)
1.923        num_tors_div

Flexible residues: A:52 A:103
Using random seed: 42

0%   10   20   30   40   50   60   70   80   90   100%
|----|----|----|----|----|----|----|----|----|----|
**************************************************

mode |   affinity | dist from best mode
     | (kcal/mol) | rmsd l.b.| rmsd u.b.
-----+------------+-

## 5.3. Custom scoring fucntion

In [20]:
open('everything.txt','wt').write('''
1.0  ad4_solvation(d-sigma=3.6,_s/q=0.01097,_c=8)  desolvation, s/q is charge dependence
1.0  ad4_solvation(d-sigma=3.6,_s/q=0.0,_c=8)  
1.0  electrostatic(i=1,_^=100,_c=8)	i is the exponent of the distance, see everything.h for details
1.0  electrostatic(i=2,_^=100,_c=8)
1.0  gauss(o=0,_w=0.5,_c=8)		o is offset, w is width of gaussian
1.0  gauss(o=3,_w=2,_c=8)
1.0  repulsion(o=0,_c=8)	o is offset of squared distance repulsion
1.0  hydrophobic(g=0.5,_b=1.5,_c=8)		g is a good distance, b the bad distance
1.0  non_hydrophobic(g=0.5,_b=1.5,_c=8)	value is linearly interpolated between g and b
1.0  vdw(i=4,_j=8,_s=0,_^=100,_c=8)	i and j are LJ exponents
1.0  vdw(i=6,_j=12,_s=1,_^=100,_c=8) s is the smoothing, ^ is the cap
1.0  non_dir_h_bond(g=-0.7,_b=0,_c=8)	good and bad
1.0  non_dir_anti_h_bond_quadratic(o=0.4,_c=8) like repulsion, but for hbond, don't use	
1.0  non_dir_h_bond_lj(o=-0.7,_^=100,_c=8)	LJ 10-12 potential, capped at ^
1.0 acceptor_acceptor_quadratic(o=0,_c=8)	quadratic potential between hydrogen bond acceptors
1.0 donor_donor_quadratic(o=0,_c=8)	quadratic potential between hydroben bond donors
1.0  num_tors_div	div constant terms are not linearly independent
1.0  num_heavy_atoms_div	
1.0  num_heavy_atoms	these terms are just added
1.0  num_tors_add
1.0  num_tors_sqr
1.0  num_tors_sqrt
1.0  num_hydrophobic_atoms
1.0  ligand_length
''');

In [21]:
!./smina.static -r rec.pdbqt -l lig_H.pdbqt --score_only --custom_scoring everything.txt --exhaustiveness 32 --num_modes 50

   _______  _______ _________ _        _______ 
  (  ____ \(       )\__   __/( (    /|(  ___  )
  | (    \/| () () |   ) (   |  \  ( || (   ) |
  | (_____ | || || |   | |   |   \ | || (___) |
  (_____  )| |(_)| |   | |   | (\ \) ||  ___  |
        ) || |   | |   | |   | | \   || (   ) |
  /\____) || )   ( |___) (___| )  \  || )   ( |
  \_______)|/     \|\_______/|/    )_)|/     \|


smina is based off AutoDock Vina. Please cite appropriately.

Weights      Terms
1            gauss(o=0,_w=0.5,_c=8)
1            gauss(o=3,_w=2,_c=8)
1            repulsion(o=0,_c=8)
1            hydrophobic(g=0.5,_b=1.5,_c=8)
1            non_hydrophobic(g=0.5,_b=1.5,_c=8)
1            vdw(i=4,_j=8,_s=0,_^=100,_c=8)
1            vdw(i=6,_j=12,_s=1,_^=100,_c=8)
1            non_dir_h_bond(g=-0.7,_b=0,_c=8)
1            non_dir_anti_h_bond_quadratic(o=0.4,_c=8)
1            non_dir_h_bond_lj(o=-0.7,_^=100,_c=8)
1            acceptor_acceptor_quadratic(o=0,_c=8)
1            donor_donor_quadratic(o=0,_c=8)
1